<a href="https://colab.research.google.com/github/asalcedo31/CSC2516_project/blob/master/CSC_2516_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import numpy as np
import torchvision
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy

In [2]:


transform = transforms.Compose(
    [transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=5,
                                         shuffle=False, num_workers=0)
_,trainset = torch.utils.data.random_split(trainset,(49500,500))
# _,trainset = torch.utils.data.random_split(trainset,(49995,5))
print(trainset.__len__())

train_data, val_data = torch.utils.data.random_split(trainset,(int(0.8*len(trainset)),int(0.2*len(trainset))))
print(train_data.__len__(),val_data.__len__() )

trainloader = torch.utils.data.DataLoader(train_data, batch_size=5,
                                          shuffle=True, num_workers=0)
valloader = torch.utils.data.DataLoader(val_data, batch_size=5,
                                          shuffle=True, num_workers=0)

# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified
500
400 100


In [0]:
image_datasets= {'train': train_data,'val': val_data}
dataloaders = {'train': trainloader, 'val': valloader}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
# class_names = image_datasets['train'].classes


In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
#                 dataloader = train_data_loader
                model.train()  # Set model to training mode
                data_idx = 0
            else:
                model.eval()   # Set model to evaluate mode
#                 dataloader = val_data_loader
                data_idx = 1

            running_loss = 0.0
            running_corrects = 0
            i=0
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
#                 print('i ',i)
#                 i+=1
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
def freeze_conv_layers(model_ft):
  children = list(model_ft.named_children())
  for name,param in children[0][1].named_parameters():
    param.requires_grad = False

In [0]:
class MaskedLinear(torch.nn.Linear):
  def __init__(self, in_features, out_features, bias=True, threshold=0.1):
    super(MaskedLinear, self).__init__(in_features,out_features)
    self.in_features = in_features
    self.out_features = out_features
    self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
    self.mask = torch.zeros(self.weight.size(), requires_grad=False).to(device)
    self.ones = torch.ones(self.weight.size(), requires_grad=False).to(device)
    self.threshold = threshold
    
    if bias:
        self.bias = nn.Parameter(torch.Tensor(out_features))
    else:
        self.register_parameter('bias', None)
    self.reset_parameters()
#   def set_threshold(self):
    
  def forward(self, input):    
    self.threshold = torch.max(torch.topk(torch.unique(self.weight),int(0.05*self.out_features*self.in_features),largest=False)[0])    
#     print(self.out_features*self.in_features)
#     print(upper[0].size())
#     torch.max(upper[0]))
    self.mask = torch.where(self.weight >= self.threshold,self.ones,self.mask)
#     print(self.weight[0:10,0:10])
#     print(mask[0:10,0:10])
#     print((self.weight*mask)[0:10,0:10])
    self.weight = torch.nn.Parameter(self.weight*self.mask).to(device)
    return F.linear(input, self.weight, self.bias)

def mask_network(network,layers_to_mask, threshold=0.002, random_init=False):
  """"
  replaces linear layers with masked linear layers
  network is the initial sequential container
  layers is a list of layers to mask
  random init is a logical indicating whether to preserve the initial weights or to modify them
  """
  for name,layer in network.named_children():   
    if int(name) in layers_to_mask:
#       print(name)
#       print(layer.in_features)
      masked_layer = MaskedLinear(layer.in_features, layer.out_features, bias=True,threshold=threshold)
      if random_init != True:
        masked_layer.weight = copy.deepcopy(layer.weight)
        masked_layer.bias = copy.deepcopy(layer.bias)
#       masked_layer.set_threshold()
      network[int(name)] = masked_layer

In [61]:
#testing code ignore
model_ft = models.vgg16(pretrained=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

mask_network(model_ft.classifier,[0,3,6])
# print(model_ft.classifier)
# print(model_ft.classifier[0].weight[120:130,120::130])
print(model_ft.classifier[0].threshold)

0.002


In [0]:
#testing code ignore
i=0
for inputs,labels in dataloaders['train']:
  inputs = inputs.to(device)
  model_ft.eval()  
  outputs = model_ft(inputs)
#   print(model_ft.classifier[3].threshold)
 

In [0]:
#baseline
model_ft = models.vgg16(pretrained=True)
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 2)

# model_ft = models.resnet18(pretrained=True)
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


In [0]:
#baseline
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

In [0]:
model_ft = models.vgg16(pretrained=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

freeze_conv_layers(model_ft)
mask_network(model_ft.classifier,[0],threshold=0.0001)

In [8]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=5)

Epoch 0/4
----------
train Loss: 4.5464 Acc: 0.0750
val Loss: 2.6614 Acc: 0.1100

Epoch 1/4
----------
train Loss: 3.5458 Acc: 0.1375
val Loss: 2.6852 Acc: 0.1600

Epoch 2/4
----------
train Loss: 3.3933 Acc: 0.1500
val Loss: 2.5406 Acc: 0.2300

Epoch 3/4
----------
train Loss: 3.6336 Acc: 0.1125
val Loss: 3.7840 Acc: 0.0300

Epoch 4/4
----------
train Loss: 3.5957 Acc: 0.1200
val Loss: 3.6383 Acc: 0.1000

Training complete in 10m 6s
Best val Acc: 0.230000


In [0]:
children = list(model_ft.named_children())
# print(list(children[0][1].named_parameters())[0])
# print(list(children[2][1].named_parameters())[0])
inpurs
print(children[2][1][0].weight)
# for name,param in children[2][1].named_parameters():
#   print(name)
#   print(param)

In [0]:
children = list(model_ft.named_children())
print(list(children[0][1].named_parameters())[0])
print(list(children[2][1].named_parameters())[0])

('0.weight', Parameter containing:
tensor([[[[-5.5373e-01,  1.4270e-01,  5.2896e-01],
          [-5.8312e-01,  3.5655e-01,  7.6566e-01],
          [-6.9022e-01, -4.8019e-02,  4.8409e-01]],

         [[ 1.7548e-01,  9.8630e-03, -8.1413e-02],
          [ 4.4089e-02, -7.0323e-02, -2.6035e-01],
          [ 1.3239e-01, -1.7279e-01, -1.3226e-01]],

         [[ 3.1303e-01, -1.6591e-01, -4.2752e-01],
          [ 4.7519e-01, -8.2677e-02, -4.8700e-01],
          [ 6.3203e-01,  1.9308e-02, -2.7753e-01]]],


        [[[ 2.3254e-01,  1.2666e-01,  1.8605e-01],
          [-4.2805e-01, -2.4349e-01,  2.4628e-01],
          [-2.5066e-01,  1.4177e-01, -5.4864e-03]],

         [[-1.4076e-01, -2.1903e-01,  1.5041e-01],
          [-8.4127e-01, -3.5176e-01,  5.6398e-01],
          [-2.4194e-01,  5.1928e-01,  5.3915e-01]],

         [[-3.1432e-01, -3.7048e-01, -1.3094e-01],
          [-4.7144e-01, -1.5503e-01,  3.4589e-01],
          [ 5.4384e-02,  5.8683e-01,  4.9580e-01]]],


        [[[ 1.7715e-01,  5.2149